In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import torch
import tensorflow as tf
from tensorflow import keras
from huggingface_hub import hf_hub_download, snapshot_download
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline
from diffusers.utils import load_image
import json

class ColorizeModel(keras.Model):
    def __init__(self, default_prompt=None, default_negative_prompt=None, torch_model_path=None, **kwargs):
        if 'torch_model_path' in kwargs:
            torch_model_path = kwargs.pop('torch_model_path')

        super().__init__(**kwargs)

        self.default_prompt = default_prompt
        self.default_negative_prompt = default_negative_prompt
        self.torch_model_path = torch_model_path

        self.prompt_embedding = keras.layers.Dense(1, use_bias=False)
        self.neg_prompt_embedding = keras.layers.Dense(1, use_bias=False)
        self.conditioning_scale = keras.layers.Dense(1, use_bias=False)
        self.guidance_scale = keras.layers.Dense(1, use_bias=False)
        self.inference_steps = keras.layers.Dense(1, use_bias=False, dtype=tf.int32)

        self.prompt_embedding.build((1, 1))
        self.neg_prompt_embedding.build((1, 1))
        self.conditioning_scale.build((1, 1))
        self.guidance_scale.build((1, 1))
        self.inference_steps.build((1, 1))

    def build(self, input_shape):
        super().build(input_shape)

    def call(self, inputs, training=False):
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({
            "default_prompt": self.default_prompt,
            "default_negative_prompt": self.default_negative_prompt,
            "torch_model_path": self.torch_model_path
        })
        return config

    def colorize(self, image_path, output_path=None, seed=42):
        """Execute image colorization using the stored PyTorch model"""
        print(f"Colorizing image using model from: {self.torch_model_path}")
        print(f"Using prompt: {self.default_prompt}")
        print(f"Using negative prompt: {self.default_negative_prompt}")

        try:
            if isinstance(image_path, str):
                print(f"Loading image from path: {image_path}")
                image = load_image(image_path)
            else:
                print("Using provided image object")
                image = image_path

            try:
                from diffusers import ControlNetModel, StableDiffusionControlNetPipeline

                print("Loading ControlNet model...")
                controlnet = ControlNetModel.from_pretrained(f"{self.torch_model_path}/controlnet")

                print("Setting up StableDiffusionControlNetPipeline...")
                pipeline = StableDiffusionControlNetPipeline.from_pretrained(
                    "runwayml/stable-diffusion-v1-5",
                    controlnet=controlnet,
                    text_encoder_pretrained_model_name_or_path=f"{self.torch_model_path}/text_encoder",
                    tokenizer_pretrained_model_name_or_path=f"{self.torch_model_path}/tokenizer",
                    unet_pretrained_model_name_or_path=f"{self.torch_model_path}/unet",
                    vae_pretrained_model_name_or_path=f"{self.torch_model_path}/vae"
                )

                pipeline.safety_checker = lambda images, **kwargs: (images, [False] * len(images))

                device = "cuda" if torch.cuda.is_available() else "cpu"
                print(f"Using device: {device}")
                pipeline = pipeline.to(device)

                guidance_scale = 1.0  # Default value
                cond_scale = 0.9     # Default control conditioning scale
                steps = 25           # Default steps

                generator = torch.Generator(device=device).manual_seed(seed)

                print("Running inference...")
                with torch.inference_mode():
                    output = pipeline(
                        prompt=self.default_prompt,
                        negative_prompt=self.default_negative_prompt,
                        image=image,
                        num_inference_steps=int(steps),
                        guidance_scale=float(guidance_scale),
                        controlnet_conditioning_scale=float(cond_scale),
                        generator=generator
                    )

                colored_image = output.images[0]

                if output_path:
                    colored_image.save(output_path)
                    print(f"Image saved to {output_path}")

                return colored_image

            except Exception as e:
                print(f"Error during PyTorch model loading or inference: {e}")
                print("This could be due to missing PyTorch components or incorrect paths")
                return None

        except Exception as e:
            print(f"Error during colorization: {e}")
            return None

def download_and_colorize(image_path, output_path="colored_output.png", username="YOUR_USERNAME", seed=60):
    repo_id = f"{username}/image-colorizer"

    print(f"Downloading model from {repo_id}...")

    model_file = hf_hub_download(repo_id=repo_id, filename="image_colorizer.h5")
    print(f"Downloaded model file to: {model_file}")

    local_dir = snapshot_download(repo_id=repo_id, local_dir="./downloaded_model")
    print(f"Downloaded model components to: {local_dir}")

    with keras.utils.custom_object_scope({'ColorizeModel': ColorizeModel}):
        print("Loading the model...")
        loaded_model = keras.models.load_model(model_file, compile=False)

        loaded_model.torch_model_path = local_dir

        try:
            with open(f"{local_dir}/config.json", "r") as f:
                config = json.load(f)
                if "default_prompt" in config:
                    loaded_model.default_prompt = config["default_prompt"]
                if "default_negative_prompt" in config:
                    loaded_model.default_negative_prompt = config["default_negative_prompt"]
        except:
            pass

        print("Executing colorization...")
        result = loaded_model.colorize(image_path, output_path=output_path, seed=seed)
        return result



In [ ]:
%%capture
grayscale_image = "/content/Grayscaleimage95886.jpeg"

username = "AmrDabour"

colorized_image = download_and_colorize(
    grayscale_image,
    output_path="colorized_result.png",
    username=username
)

try:
    from IPython.display import display
    display(colorized_image)
except:
    print("Image saved to colorized_result.png")

The config attributes {'dropout': 0.0, 'sample_size': 32} were passed to ControlNetModel, but are not expected and will be ignored. Please verify your config.json configuration file.
Keyword arguments {'text_encoder_pretrained_model_name_or_path': '/content/downloaded_model/text_encoder', 'tokenizer_pretrained_model_name_or_path': '/content/downloaded_model/tokenizer', 'unet_pretrained_model_name_or_path': '/content/downloaded_model/unet', 'vae_pretrained_model_name_or_path': '/content/downloaded_model/vae'} are not expected by StableDiffusionControlNetPipeline and will be ignored.
